# Defining Test and Train Paths

In [2]:
import os.path

test_path = r'../data/test'
train_path = '../data/train/'

# Finding the height, width and channels of all training images

In [3]:
import PIL.Image as Image

width = list()
height = list()

for category in os.listdir(train_path):

    if category.find('.') != 0:
        for img in os.listdir(os.path.join(train_path, category)):
            if img.find('.') != 0:
                img_path = os.path.join(train_path, category, img)
                img = Image.open(img_path)
                width.append(img.width)
                height.append(img.height)

# Deducing Image Shape

In [18]:
import numpy as np

# Image shape will be an average of individual width/height of all training images
image_shape = (int(np.round(np.mean(width))), int(np.round(np.mean(height))), 3)
image_shape

(778, 512, 3)

# Using Image Data Generator

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(
    rotation_range=20,  # Rotate images randomly by 20 degrees
    width_shift_range=0.1,  # Shift width of the image by 10%
    height_shift_range=0.1,
    rescale=1 / 255,  # Normalize images
    shear_range=0.1,  # Cutting away 10% of the image
    zoom_range=0.1,  # Zoom in 10%
    horizontal_flip=True,
    fill_mode='nearest'  # Read documentation
)

# Making classes Known to the Generator

In [20]:
image_generator.flow_from_directory(train_path)

image_generator.flow_from_directory(test_path)

Found 7 images belonging to 3 classes.
Found 7 images belonging to 3 classes.


# Creating the Model

In [21]:
from tensorflow.keras.applications import ResNet50

In [22]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=image_shape
)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(base_model)
model.add(GlobalMaxPooling2D(name="gap"))

#avoid over-fitting
model.add(Dropout(rate=0.2, name="dropout_out"))

# Set NUMBER_OF_CLASSES to the number of your final predictions.
model.add(Dense(3, activation="softmax", name="fc_out"))
base_model.trainable = False

# Defining Batch Size

In [24]:
batch_size = 3

# Test and Train Generators

In [25]:
train_generator = image_generator.flow_from_directory(
    train_path,
    # All images will be resized to target height and width.
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='rgb',
    # Since we use categorical cross-entropy loss, we need categorical labels
    class_mode="categorical"
)

Found 7 images belonging to 3 classes.


In [26]:
train_generator.class_indices

{'college': 0, 'mall': 1, 'office': 2}

In [33]:
test_generator = image_generator.flow_from_directory(
    test_path,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

Found 7 images belonging to 3 classes.


In [34]:
model.compile(
    'adam',
    'categorical_crossentropy',
    metrics=['accuracy']
)

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 25, 16, 2048)      23587712  
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 2048)              0         
_________________________________________________________________
dropout_out (Dropout)        (None, 2048)              0         
_________________________________________________________________
fc_out (Dense)               (None, 3)                 6147      
Total params: 23,593,859
Trainable params: 6,147
Non-trainable params: 23,587,712
_________________________________________________________________


# Training the Model

In [36]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [38]:
results = model.fit_generator(
    train_generator,
    epochs=2,
    validation_data=test_generator,
    callbacks=[early_stop],
)

Epoch 1/2
3/3 [==============================] - 87s 29s/step - loss: 12.2805 - accuracy: 0.2857 - val_loss: 12.5363 - val_accuracy: 0.2857
Epoch 2/2
3/3 [==============================] - 92s 31s/step - loss: 12.2805 - accuracy: 0.2857 - val_loss: 8.9545 - val_accuracy: 0.2857


# Saving Model's Weights

In [ ]:
from tensorflow.keras.models import save_model

save_model(model, '../models/resnet50_model.h5')

# Evaluating Model's Performance

In [39]:
model.evaluate(test_generator)

3/3 [==============================] - 19s 6s/step - loss: 20.6106 - accuracy: 0.2857


[20.610580444335938, 0.2857143]

In [84]:
# Make predictions
predictions = model.predict_generator(test_generator)

In [87]:
# SETTING CUT-OFF PROBABILITY
predictions = predictions > 0.5

In [88]:
predictions

array([[False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True]])

In [90]:
test_generator.classes

array([0, 0, 1, 1, 1, 2, 2], dtype=int32)

In [93]:
one_hot_encoded_y_test = [
    [1, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 1, 0],
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 1],
]

In [98]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(one_hot_encoded_y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         3
           2       0.29      1.00      0.44         2

   micro avg       0.29      0.29      0.29         7
   macro avg       0.10      0.33      0.15         7
weighted avg       0.08      0.29      0.13         7
 samples avg       0.29      0.29      0.29         7



In [99]:
import jovian

jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "justvikram/exploring-models" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/justvikram/exploring-models


'https://jovian.ai/justvikram/exploring-models'